<a href="https://colab.research.google.com/github/4Kaustubh/Project-Curvetopia/blob/main/Project_Curvetopia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install scipy scikit-learn svgwrite cairosvg

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import svgwrite
import cairosvg
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from scipy.spatial import ConvexHull
from sklearn.decomposition import PCA


def read_csv(csv_path):
    np_path_XYs = np.genfromtxt(csv_path, delimiter=',')
    path_XYs = []
    for i in np.unique(np_path_XYs[:, 0]):
        npXYs = np_path_XYs[np_path_XYs[:, 0] == i][:, 1:]
        XYs = []
        for j in np.unique(npXYs[:, 0]):
            XY = npXYs[npXYs[:, 0] == j][:, 1:]
            XYs.append(XY)
        path_XYs.append(XYs)
    return path_XYs

def regularize_shapes(paths_XYs):
    regularized_paths = []
    for XYs in paths_XYs:
        smoothed_XYs = []
        for XY in XYs:

            t = np.linspace(0, 1, len(XY))
            t_interp = np.linspace(0, 1, len(XY) * 10)
            XY_interp = np.column_stack([np.interp(t_interp, t, XY[:, 0]), np.interp(t_interp, t, XY[:, 1])])


            smoothed_XY = savgol_filter(XY_interp, window_length=11, polyorder=3, axis=0)
            smoothed_XYs.append(smoothed_XY)
        regularized_paths.append(smoothed_XYs)
    return regularized_paths


def close_paths(paths_XYs):
    closed_paths = []
    for XYs in paths_XYs:
        closed_XYs = []
        for XY in XYs:
            if not np.allclose(XY[0], XY[-1]):
                XY = np.vstack([XY, XY[0]])
            closed_XYs.append(XY)
        closed_paths.append(closed_XYs)
    return closed_paths


def generate_colors(num_shapes):
    colors = plt.cm.get_cmap('tab10', num_shapes).colors
    return colors


def paths_to_svg(paths_XYs, svg_path):
    W, H = 0, 0
    for path_XYs in paths_XYs:
        for XY in path_XYs:
            W, H = max(W, np.max(XY[:, 0])), max(H, np.max(XY[:, 1]))

    padding = 0.1
    W, H = int(W + padding * W), int(H + padding * H)


    dwg = svgwrite.Drawing(svg_path, profile='tiny', size=(W, H), shape_rendering='crispEdges')
    group = dwg.g()

    colors = generate_colors(len(paths_XYs))

    for i, path_XYs in enumerate(paths_XYs):
        path_data = []
        color = svgwrite.rgb(255, 0, 0)
        for XY in path_XYs:
            path_data.append(f"M{XY[0, 0]},{XY[0, 1]}")
            path_data += [f"L{x},{y}" for x, y in XY[1:]]
            if np.allclose(XY[0], XY[-1]):
                path_data.append("Z")
        group.add(dwg.path(d=" ".join(path_data), fill='none', stroke=color, stroke_width=1))

    dwg.add(group)
    dwg.save()


    png_path = svg_path.replace('.svg', '.png')
    cairosvg.svg2png(url=svg_path, write_to=png_path, output_width=W, output_height=H, background_color='white')


def process_shapes(csv_path, svg_output_path):
    paths_XYs = read_csv(csv_path)
    paths_XYs = regularize_shapes(paths_XYs)
    paths_XYs = close_paths(paths_XYs)
    paths_to_svg(paths_XYs, svg_output_path)


csv_file_path = '/content/input.csv'
svg_output_path = '/content/output.svg'
process_shapes(csv_file_path, svg_output_path)

<ipython-input-10-441c8eeb287e>:53: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap('tab10', num_shapes).colors
